# this is the sequential execution of the input data file to disect every step

In [1]:
"""Routine for decoding the digits-10 binary file format."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from os import listdir
from os.path import isfile, join

from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

# Process images of this size. Note that this differs from the original CIFAR
# image size of 32 x 32. If one alters this number, then the entire model
# architecture will change and any model would need to be retrained.
IMAGE_SIZE = 24

# Global constants describing the CIFAR-10 data set.
NUM_CLASSES = 10
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000 # was set from 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000

import re
import sys
import tarfile

from six.moves import urllib

# FLAGS = tf.app.flags.FLAGS

# # Basic model parameters.
# #former batch size was 128
# tf.app.flags.DEFINE_integer('batch_size', 1 ,
#                             """Number of images to process in a batch.""")
# tf.app.flags.DEFINE_string('data_dir', './tmp/cifar10_data',
#                            """Path to the CIFAR-10 data directory.""")
# tf.app.flags.DEFINE_boolean('use_fp16', False,
#                             """Train the model using fp16.""")
# tf.app.flags.DEFINE_string('eval_data', 'test',
#                  """Either 'test' or 'train_eval'.""")

batch_size = 2

data_dir = './data_format_2/digits-10-batches-bin'

use_fp16 = False

eval_data = 'test'



# Constants describing the training process.
MOVING_AVERAGE_DECAY = 0.9999     # The decay to use for the moving average.
NUM_EPOCHS_PER_DECAY = 350.0      # Epochs after which learning rate decays.
LEARNING_RATE_DECAY_FACTOR = 0.1  # Learning rate decay factor.
INITIAL_LEARNING_RATE = 0.1       # Initial learning rate.

# If a model is trained with multiple GPUs, prefix all Op names with tower_name
# to ifferentiate the operations. Note that this prefix is removed from the
# names of the summaries when visualizing a model.
TOWER_NAME = 'tower'

DATA_URL = 'https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz'


In [2]:
filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i)
                     for i in xrange(1, 5)]

filename_queue = tf.train.string_input_producer(filenames)

# Dimensions of the images in the CIFAR-10 dataset.
# See http://www.cs.toronto.edu/~kriz/cifar.html for a description of the
# input format.
label_bytes = 1  # 2 for CIFAR-100
height = 32
width = 32
depth = 3
image_bytes = height * width * depth
# Every record consists of a label followed by the image, with a
# fixed number of bytes for each.
record_bytes = label_bytes + image_bytes
reader = tf.FixedLengthRecordReader(record_bytes=record_bytes)


# Read a record, getting filenames from the filename_queue.  No
# header or footer in the CIFAR-10 format, so we leave header_bytes
# and footer_bytes at their default of 0.


In [32]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    key, value = reader.read(filename_queue)

    # Convert from a string to a vector of uint8 that is record_bytes long.
    record_bytes = tf.decode_raw(value, tf.uint8)

    # The first bytes represent the label, which we convert from uint8->int32.
    # The remaining bytes after the label represent the image, which we reshape
    # from [depth * height * width] to [depth, height, width].
    #depth_major = tf.strided_slice(record_bytes, [1],[1 + image_bytes])
    #label = tf.strided_slice(record_bytes, [0], [1])

#     # Convert from [depth, height, width] to [height, width, depth].
#     uint8image = tf.transpose(depth_major, [1, 2, 0])
    #print(sess.run(depth_major))
    print(sess.run(record_bytes[0]))
#     print(sess.run(depth_major)) # data/heart.csv:2
#     print(sess.run(label)) # 144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1


    coord.request_stop()
    coord.join(threads)

0


# Below two blocks contains code reading .mat (matlab) files into binary format for tensorflow

the code uses scipy and numpy to read, reformat and save data into binary fixed length records used by tensorflow

In [37]:
## generating the training batches
import numpy as np
import scipy.io
from array import array

input_file = '/home/olle/PycharmProjects/Diabetic_Retinopathy_Detection/data/train/train.zip.005'
read_input = scipy.io.loadmat(input_file)
j=0
output_file = open('data_batch_%d.bin' % j, 'ab')

#X = read_input['X'].reshape(32*32*3, read_input['X'].shape[3])

for i in range(0, 64000):

    # create new bin file
    if i>0 and i % 12800 == 0:
        output_file.close()
        j=j+1
        output_file = open('data_batch_%d.bin' % j, 'ab')

    # Write to bin file
    if read_input['y'][i] == 10:
        read_input['y'][i] = 0
    read_input['y'][i].astype('uint8').tofile(output_file)
    read_input['X'][:,:,:,i].astype('uint32').tofile(output_file)

output_file.close()

In [ ]:
## generating the training batches
import numpy as np
import scipy.io
from array import array

test_input = scipy.io.loadmat('data_format_2/test_32x32.mat')
j=0
output_testfile = open('test_batch.bin', 'ab')

X_test = test_input['X'].reshape(test_input['X'].shape[3], 32*32*3)
for i in range(0,26032):
    
    # Write to bin file
    if test_input['y'][i] == 10:
        test_input['y'][i] = 0
    test_input['y'][i].astype('uint8').tofile(output_testfile)
    X_test[i].astype('uint32').tofile(output_testfile)

output_testfile.close()

# How to use below code chunk
below is a test script for input data up until batch ops. The idea is that the below script should implement 
1. file parsing, 
2. create a suitable reader (here fixed length but can be changed to TFrecord,
3. decoder of data type into numeric values
4. Then open a session that extract features and labels from data
5. pre-processes the data
6. within the session easily prints the values such that debugging can be made
                             
Finally the below steps can be inserted into the CIFAR turorial design priciples for loading data. The code extracts the numeric values into numpy format, the code can be easily extended to vizualize the numeric data as pictures again using e.g. Pillow.

In [ ]:
#setting parameters for lading data and shape parameters for preprocesing
label_bytes = 1  # 2 for CIFAR-100
height = 32
width = 32
depth = 3

#might have to be made dynamic
image_bytes = height * width * depth

# Every record consists of a label followed by the image, with a
# fixed number of bytes for each.
record_bytes = label_bytes + image_bytes

filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i)
                     for i in xrange(1, 5)]

filename_queue = tf.train.string_input_producer(filenames)
reader = tf.FixedLengthRecordReader(record_bytes=record_bytes)

key, value = reader.read(filename_queue)
record_bytes = tf.decode_raw(value, tf.uint8)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    label = tf.cast(tf.strided_slice(record_bytes, [0], [label_bytes]), tf.int32)

    depth_major = tf.reshape(tf.strided_slice(record_bytes, [label_bytes], [label_bytes + image_bytes]),
        [depth, height, width])

    # Convert from [depth, height, width] to [height, width, depth].
    uint8image = tf.transpose(depth_major, [1, 2, 0])
    reshaped_image = tf.cast(uint8image, tf.float32)

    # Image processing for evaluation.
    # Crop the central [height, width] of the image.
    resized_image = tf.image.resize_image_with_crop_or_pad(reshaped_image,
                                                           height, width)

    # Subtract off the mean and divide by the variance of the pixels.
    float_image = tf.image.per_image_standardization(resized_image)
    
    # Set the shapes of tensors.
    float_image.set_shape([height, width, 3])
    label.set_shape([1])
    
    # minimum number elements in the queue after a dequeue, used to ensure
    # that the samples are sufficiently mixed
    
    # I think 10 times the BATCH_SIZE is sufficient   
    #checkpoint - see so image has required form
    for i in range(0,100):
        print(sess.run(record_bytes)[0]) # data/heart.csv:2
    #print(sess.run(label)) # 144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1

    coord.request_stop()
    coord.join(threads)